In [42]:
import pandas as pd
import numpy as np
import plotly.express as px

In [18]:
df = pd.read_csv('../data/nlf_mock_drafts_23.csv')

In [19]:
num_filter = df.groupby(['player'])['mock_draft'].count().reset_index(name='num_occurrences')
avg_pos_filter = df.groupby(['player'])['pick_number'].mean().reset_index(name='adp')

In [20]:
df = pd.merge(df, num_filter, how='left', on='player')
df = pd.merge(df, avg_pos_filter, how='left', on='player')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 0 to 309
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mock_draft       310 non-null    object 
 1   authors          310 non-null    object 
 2   player           310 non-null    object 
 3   pick_number      310 non-null    int64  
 4   last_updated     10 non-null     object 
 5   url              160 non-null    object 
 6   num_occurrences  310 non-null    int64  
 7   adp              310 non-null    float64
dtypes: float64(1), int64(2), object(5)
memory usage: 29.9+ KB


In [51]:
composite = df[df['num_occurrences']>=3].groupby(['player'])['adp'].mean().sort_values()[:31].reset_index()

In [60]:
top_first = list(composite['player'])

In [52]:
composite['pick_number'] = composite['adp'].rank().astype('int')

In [53]:
composite['mock_draft'] = 'Composite ADP'
composite['authors'] = 'Composite'
composite['last_updated'] = '04/25'
composite['url'] = np.nan
composite['num_occurrences'] = np.nan

In [56]:
df = pd.concat([composite[['mock_draft', 'authors', 'player', 'pick_number', 'last_updated', 'url', 'num_occurrences', 'adp']],
          df])

In [63]:
df = df[df.player.isin(top_first)]

In [105]:
df_heatmap = pd.pivot_table(df, 
                            index=['player', 'adp'], 
                            columns='pick_number',
                            values = 'mock_draft', 
                            aggfunc='count',
                            fill_value=0).reset_index()

In [106]:
df_heatmap = df_heatmap.sort_values(by='adp')

In [109]:
df_heatmap = df_heatmap.drop(columns='adp')

In [110]:
df_heatmap = df_heatmap.set_index('player')

In [124]:
fig = px.imshow(df_heatmap,
                color_continuous_scale=['#FFFFFF', '#0085CA'])

fig.update_layout(autosize=False, 
                  showlegend=False,
                  width=900,
                  height=800)

fig.update_xaxes(side='top')

fig.update_traces(showlegend=False)

fig.show()

In [131]:
df.groupby(['player'])['pick_number'].var().sort_values(ascending=False)

player
Bijan Robinson        52.054545
Christian Gonzalez    47.618182
Nolan Smith           39.000000
Brian Branch          37.000000
Myles Murphy          36.666667
Will Levis            35.333333
Anthony Richardson    34.690909
Darnell Wright        32.446970
Dalton Kincaid        29.818182
Bryan Bresee          27.952381
Calijah Kancey        27.285714
O'Cyrus Torrence      27.000000
Michael Mayer         25.111111
Emmanuel Forbes       24.400000
Jaxon Smith-Njigba    22.690909
Joey Porter Jr        21.963636
Jordan Addison        19.433333
Quentin Johnston      14.476190
Peter Skoronski       14.218182
Paris Johnson Jr      12.854545
Zay Flowers           12.400000
Jahmyr Gibbs          12.333333
Lukas Van Ness        12.054545
Deonte Banks          10.055556
CJ Stroud              9.854545
Jalen Carter           6.254545
Devon Witherspoon      5.363636
Tyree Wilson           4.818182
Will Anderson Jr       4.672727
Broderick Jones        1.854545
Bryce Young            0.090909
N

In [134]:
df.groupby(['pick_number'])['player'].nunique().sort_values(ascending=False)

pick_number
10    10
20     9
28     8
24     8
23     8
9      8
13     7
17     7
21     7
22     7
14     7
16     7
12     7
11     7
8      7
27     7
18     7
25     6
15     6
19     6
7      6
2      5
6      5
29     5
30     4
5      4
26     4
4      4
3      4
1      2
31     2
Name: player, dtype: int64